原始代码： https://colab.research.google.com/drive/1Uyz2Wsm0pUQWuZ0t6DnBUm4mnmqUERLh#scrollTo=ISwXERZACOsz

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
import pandas as pd
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import json
import os
import sys
import zipfile

## 数据集：

In [3]:
%%writefile config_train.json
{
  "samplewise_std_normalization" : true,
  "samplewise_center"  : true,
  "input_image_format" : "jpg",
  "input_csv_file"     : "snap-training-dataset.csv",
  "category"           : "H",
  "prc_lower_withheld": 0,
  "prc_upper_withheld": 0,

  "horizontal_flip"    : false,
  "vertical_flip"      : false,
  "rotation_range"     : 10,
  "width_shift_range"  : 0.1,
  "height_shift_range" : 0.1,
  "shear_range"        : 0.05,
  "zoom_range"         : 0.2,
  "fill_mode"          : "reflect",

  "img_size"           : 128,
  "num_epochs"         : 200,
  "test_size"          : 0.4,
  "dropout_rate"       : 0.5,
  "epsilon"            : 0.0001,
  "min_lr"             : 0.0001,
  "factor"             : 0.8
}

Overwriting config_train.json


In [6]:
with open('config_train.json') as f:
    config = json.load(f)

# config variables
imsize    = int(config["img_size"])
num_epochs = int(config["num_epochs"])
test_size = float(config["test_size"])
height_shift_range = float(config["height_shift_range"])
width_shift_range = float(config["width_shift_range"])
rotation_range = float(config["rotation_range"])
samplewise_std_normalization = config["samplewise_std_normalization"]
horizontal_flip = config["horizontal_flip"]
vertical_flip = config["vertical_flip"]
samplewise_center = config["samplewise_center"]
shear_range = float(config["shear_range"])
zoom_range = float(config["zoom_range"])
dropout_rate = float(config["dropout_rate"])
epsilon = float(config["epsilon"])
min_lr = float(config["min_lr"])
factor = float(config["factor"])
input_image_format = config["input_image_format"]
input_csv_file = config["input_csv_file"]
category = config["category"]
fill_mode = config["fill_mode"]
prc_lower_withheld = config['prc_lower_withheld']
prc_upper_withheld = config['prc_upper_withheld']
IMG_SIZE = (imsize, imsize)

In [4]:
!wget https://github.com/dbuscombe-usgs/OpticalWaveGauging_DNN/raw/master/snap-training-dataset.csv

--2024-02-19 22:53:12--  https://github.com/dbuscombe-usgs/OpticalWaveGauging_DNN/raw/master/snap-training-dataset.csv
Connecting to 127.0.0.1:7890... connected.
Proxy request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/OpticalWaveGauging/OpticalWaveGauging_DNN/raw/master/snap-training-dataset.csv [following]
--2024-02-19 22:53:13--  https://github.com/OpticalWaveGauging/OpticalWaveGauging_DNN/raw/master/snap-training-dataset.csv
Reusing existing connection to github.com:443.
Proxy request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/OpticalWaveGauging/OpticalWaveGauging_DNN/master/snap-training-dataset.csv [following]
--2024-02-19 22:53:14--  https://raw.githubusercontent.com/OpticalWaveGauging/OpticalWaveGauging_DNN/master/snap-training-dataset.csv
Connecting to 127.0.0.1:7890... connected.
Proxy request sent, awaiting response... 200 OK
Length: 43795 (43K) [text/plain]
Saving to: ‘snap-training-dataset.csv.1’

sna

In [7]:
def get_and_tidy_df(base_dir, input_csv_file, image_dir, category):
	'''复制原数据集，使容量为原来的5倍'''
	df = pd.read_csv(os.path.join(base_dir, input_csv_file))
	df['path'] = df['id'].map(lambda x: os.path.join(base_dir,
		                                                image_dir,'{}'.format(x)))

	df = df.rename(index=str, columns={" H": "H", " T": "T"})

	if category == 'H':
		mean = df['H'].mean()
		div = df['H'].std()
		df['zscore'] = df['H'].map(lambda x: (x-mean)/div)
	elif category == 'T':
		mean = df['T'].mean()
		div = df['T'].std()
		df['zscore'] = df['T'].map(lambda x: (x-mean)/div)
	else:
		print("Unknown category: "+str(category))
		print("Fix config file, exiting now ...")
		sys.exit()

	df.dropna(inplace = True)
	try:
		df = df.sort_values(by='time', axis=0)
	except:
		df = df.sort_values(by='id', axis=0)

	if category == 'H':
		df['category'] = pd.cut(df['H'], 10)
	else:
		df['category'] = pd.cut(df['T'], 8)

	df['index1'] = df.index; new_df = df.groupby(['category']).apply(lambda x: x.sample(int(len(df)/2), replace = True)).reset_index(drop = True)

	return new_df, df

In [8]:
# call the utils.py function get_and_tidy_df
image_dir = 'snap_images/data'
new_df, df = get_and_tidy_df(os.getcwd(), input_csv_file, image_dir, category)

/tmp/ipykernel_145503/3895373730.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['index1'] = df.index; new_df = df.groupby(['category']).apply(lambda x: x.sample(int(len(df)/2), replace = True)).reset_index(drop = True)
/tmp/ipykernel_145503/3895373730.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['index1'] = df.index; new_df = df.groupby(['category']).apply(lambda x: x.sample(int(len(df)/2), replace = True)).reset_index(drop = True)


In [9]:
print('New Data Size:', new_df.shape[0], 'Old Size:', df.shape[0])

train_df, valid_df = train_test_split(new_df,
                    test_size = test_size,
                    random_state = 2018,
                    stratify = new_df['category'])
print('train', train_df.shape[0], 'validation', valid_df.shape[0])

New Data Size: 4900 Old Size: 980
train 2940 validation 1960


In [10]:
if category=="H":
  idx_label = 2
elif category=="T":
  idx_label = 3

In [11]:
class WaveHeightDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 4]
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        label = self.dataframe.iloc[idx, idx_label]

        if self.transform:
            image = self.transform(image)

        return image, label

In [12]:
# 数据转换与增强
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # 调整图像大小
    # transforms.RandomHorizontalFlip(),  # 水平翻转
    # transforms.RandomVerticalFlip(),  # 垂直翻转
    transforms.RandomRotation(10),  # 随机旋转
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), shear=shear_range, scale=(0.8, 1.2)),  # 位移、剪切和缩放
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # 归一化
])

# 数据加载
train_dataset = WaveHeightDataset(train_df, transform=transform)
test_dataset = WaveHeightDataset(valid_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [13]:
train_df.iloc[0, :]

id                                     1516464000.cx.snap.jpg
time                                               1516464000
H                                                        1.79
T                                                       14.15
path        /home/ubuntu/Documents/OpticalWaveGauging_DNN/...
zscore                                                1.91159
category                                       (1.692, 1.909]
index1                                                    422
Name: 3047, dtype: object

In [14]:
class OWGNet(nn.Module):
    def __init__(self):
        super(OWGNet, self).__init__()
        self.mobilenet = models.mobilenet_v2(pretrained=False)
        self.mobilenet.features[0][0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(1280, 1)
        )

    def forward(self, x):
        x = self.mobilenet.features(x)
        x = nn.functional.adaptive_avg_pool2d(x, 1)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: " + str(device))

model = OWGNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().view(-1, 1)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(train_loader)


device: cuda


/home/ubuntu/anaconda3/envs/torch220/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/anaconda3/envs/torch220/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [16]:
# 指定保存模型的路径
checkpoint_path = 'checkPoints/model_checkpoint.pth'

In [18]:
num_epochs = 100  # 根据需要调整

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss}')
    
    # 在每个epoch后保存模型checkpoint
    # 你可以选择保存更多的信息，如优化器的状态
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': train_loss,
    }, checkpoint_path)


Epoch 1, Train Loss: 0.21533022869540297
Epoch 2, Train Loss: 0.15933393508843754
Epoch 3, Train Loss: 0.1369465592560237
Epoch 4, Train Loss: 0.116467413948039
Epoch 5, Train Loss: 0.10425567226079495
Epoch 6, Train Loss: 0.09378270622666764
Epoch 7, Train Loss: 0.09128506657788935
Epoch 8, Train Loss: 0.08420567387593506
Epoch 9, Train Loss: 0.08307803351351101
Epoch 10, Train Loss: 0.07039195521856131
Epoch 11, Train Loss: 0.07051782941182508
Epoch 12, Train Loss: 0.06459084591504348
Epoch 13, Train Loss: 0.06202148741273129
Epoch 14, Train Loss: 0.059925824616347316
Epoch 15, Train Loss: 0.05940231788174614
Epoch 16, Train Loss: 0.05287109287050755
Epoch 17, Train Loss: 0.04995584431225839
Epoch 18, Train Loss: 0.0432764986054162
Epoch 19, Train Loss: 0.05160599218114563
Epoch 20, Train Loss: 0.04867023049408327
Epoch 21, Train Loss: 0.04418586724458019
Epoch 22, Train Loss: 0.04557632242390157
Epoch 23, Train Loss: 0.043489348381231335
Epoch 24, Train Loss: 0.04443321648843424
Epo

In [ ]:
# 加载模型checkpoint
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

# 在恢复训练之前，确保模型处于训练模式
model.train()
# 接着可以继续训练或进行评估...

In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()  # 设置模型为评估模式
    predictions = []
    actuals = []
    with torch.no_grad():  # 在评估阶段不计算梯度
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().view(-1, 1)
            outputs = model(images)
            predictions.extend(outputs.view(-1).cpu().numpy())  # 将预测保存起来
            actuals.extend(labels.view(-1).cpu().numpy())  # 保存真实标签
    return actuals, predictions


In [ ]:
actuals, predictions = evaluate_model(model, test_loader, device)

# 绘制实际值与预测值的对比图
plt.figure(figsize=(10, 6))
plt.scatter(actuals, predictions, alpha=0.5)
plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'r')  # 绘制理想线
plt.xlabel('Actual Wave Height (m)')
plt.ylabel('Predicted Wave Height (m)')
plt.title('Actual vs. Predicted Wave Heights')
plt.show()


In [ ]:
def show_predictions(test_loader, model, device, num_images=5):
    model.eval()
    images, labels = next(iter(test_loader))
    fig, axes = plt.subplots(1, num_images, figsize=(20, 4))
    with torch.no_grad():
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        for i in range(num_images):
            ax = axes[i]
            img = images[i].cpu().squeeze()  # 移除批处理维度
            label = labels[i].item()
            prediction = outputs[i].item()
            ax.imshow(img.numpy(), cmap='gray')
            ax.set_title(f'True: {label:.3f}\nPred: {prediction:.3f}')
            ax.axis('off')
    plt.show()

# 显示一些预测结果
show_predictions(test_loader, model, device, num_images=5)
